# Serve Zone Distribution

function to classify zones of each serve and calculate win percentage in each zone

creates a json file that can be passed into analytics html to create visual

In [1]:
import pandas as pd
import numpy as np

In [2]:
def classify_zone(df):
    x = df['x_coord']
    y = df['y_coord']
    sign = x * y # if sign is pos, it's on ad side, if neg, it's deuce

    if (x < -105) or (x > 105):
        if sign > 0:
            return 'Ad Wide'
        else:
            return 'Deuce Wide'
    elif (-105 <= x <= -52.5) or (52.5 <= x <= 105):
        if sign > 0:
            return 'Ad Body'
        else:
            return 'Deuce Body'
    elif -52.5 < x < 52.5:
        if sign > 0:
            return 'Ad T'
        else:
            return 'Deuce T'
    else:
        return np.nan

In [3]:
def serve_zone_distribution(file_path):
    df_shots = pd.read_excel(file_path, sheet_name='Shots')
    df_points = pd.read_excel(file_path, sheet_name='Points')
    
    # only use matches with complete data
    df_shots = df_shots[df_shots['__source_file__'].isin(df_points['__source_file__'])]

    # add column for winner of the point
    combined = pd.merge(df_shots, df_points[['Point', 'Game', 'Set', 'Point Winner', '__source_file__']], on=['Point', 'Game', 'Set', '__source_file__'], how='left')
    
    serves = combined[combined['Stroke'] == 'Serve']
    serves_in = serves[serves['Result'] == 'In']

    # zone classification
    serves_in['x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
    serves_in['y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
    serves_in['Zone'] = serves_in.apply(classify_zone, axis=1)

    # get win proportions and convert to json
    zones = serves_in.groupby('Zone')['Point Winner'].value_counts().unstack()
    zones['Win Proportion'] = zones['host'].astype(str) + '/' + (zones['host'] + zones['guest']).astype(str)
    zones.reset_index(inplace=True)
    zones.columns.name = None
    zones = zones[['Zone', 'Win Proportion']]
    zones.to_json('serve_dist.json', orient='records')

In [4]:
path = '../../data/mens/Rudy Quan/combined.xlsx'
serve_zone_distribution(path)

/var/folders/ld/ry50f5bs3tzgqgpbq7z2_6pw0000gn/T/ipykernel_70130/4243561219.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serves_in['x_coord'] = serves_in['Bounce (x)'] * 38.2764654418
/var/folders/ld/ry50f5bs3tzgqgpbq7z2_6pw0000gn/T/ipykernel_70130/4243561219.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  serves_in['y_coord'] = (serves_in['Bounce (y)'] - 11.8872) * 38.2764654418
/var/folders/ld/ry50f5bs3tzgqgpbq7z2_6pw0000gn/T/ipykernel_70130/4243561219.py:17: SettingWithCopyWarning: 
A value 